In [2]:
# !pip install xgboost
# !pip install seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


In [3]:
import os
import re
import sys
import math
import json
import time
import joblib
import pickle
import tarfile
import warnings
import boto3
import botocore
import sagemaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

from IPython.display import Image
from IPython.display import display
from IPython.display import FileLink, FileLinks
from platformdirs import site_config_dir, user_config_dir
from time import gmtime, strftime
from pprint import pprint

from sagemaker import Session
from sagemaker import get_execution_role
from sagemaker import image_uris, model_uris, script_uris
from sagemaker.experiments.run import Run, load_run
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.spark.processing import PySparkProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.debugger import Rule, ProfilerRule, rule_configs
from sagemaker.remote_function import remote, RemoteExecutor
from sagemaker.analytics import HyperparameterTuningJobAnalytics
from sagemaker.tuner import (
    CategoricalParameter, ContinuousParameter, IntegerParameter,
    HyperparameterTuner, WarmStartConfig, WarmStartTypes
)

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [4]:
print(pd.__version__)
print(xgb.__version__)
print(sagemaker.__version__)

2.1.4
1.7.6
2.214.3


In [5]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 30)

warnings.filterwarnings("ignore")

In [6]:
boto_session = boto3.Session()
client_sagemaker = boto_session.client("sagemaker")
client_s3 = boto_session.client("s3")
sagemaker_session = sagemaker.Session()
# client_sagemaker = sagemaker_session.sagemaker_client
region_name = boto_session.region_name
sagemaker_role = sagemaker.get_execution_role()

In [15]:
%store -r 

%store

Stored variables and their in-db values:
bucket_name                         -> 'sagemaker-us-east-1-891377318910'
bucket_prefix                       -> 'from-idea-to-production/xgboost'
col_target                          -> 'y'
dataset_file_local_path             -> 'data/bank-additional/bank-additional-full.csv'
dataset_raw                         -> 'bank-additional-full.csv'
domain_id                           -> 'd-ehxji4qaadry'
experiment_name                     -> 'itau-experiment-2024-10-13-18-09-47'
initialized                         -> True
input_s3_url                        -> 's3://sagemaker-us-east-1-891377318910/workshop_v2
loca_transformed_path               -> './data/transformed'
local_prefix                        -> './data/raw'
output_s3_url                       -> 's3://sagemaker-us-east-1-891377318910/workshop_v2
region                              -> 'us-east-1'
region_name                         -> 'us-east-1'
s3_data_raw_prefix                  -> 'data/

In [11]:
# client_s3.download_file(bucket_name, f"{key_prefix}/train/train.csv", local_train_data)
# client_s3.download_file(bucket_name, f"{key_prefix}/validation/validation.csv", local_validation_data)
# client_s3.download_file(bucket_name, f"{key_prefix}/test/test_x.csv", local_testx_data)
# client_s3.download_file(bucket_name, f"{key_prefix}/test/test_y.csv", local_testy_data)

In [8]:
local_train_data = f"{loca_transformed_path}/train.csv"
local_validation_data = f"{loca_transformed_path}/validation.csv"
local_testx_data = f"{loca_transformed_path}/test_x.csv"
local_testy_data = f"{loca_transformed_path}/test_y.csv"

key_prefix = f"{s3_prefix}/data/transformed"

In [9]:
# train_data = pd.read_csv(local_train_data, engine="pyarrow", dtype_backend="pyarrow", header=None)
# validation_data = pd.read_csv(local_validation_data, engine="pyarrow", dtype_backend="pyarrow", header=None)
# test_data = pd.read_csv(local_test_data, engine="pyarrow", dtype_backend="pyarrow", header=None)

# train_data = np.genfromtxt(local_train_data, delimiter=',', names=None)
# train_data = np.genfromtxt(local_train_data, delimiter=',', names=None)
# train_data = np.genfromtxt(local_train_data, delimiter=',', names=None)

# train_data = pd.read_csv(local_train_data, engine="pyarrow", dtype_backend="pyarrow")
# validation_data = pd.read_csv(local_validation_data, engine="pyarrow", dtype_backend="pyarrow")
# test_data = pd.read_csv(local_test_data, engine="pyarrow", dtype_backend="pyarrow")

train_data = pd.read_csv(local_train_data)
validation_data = pd.read_csv(local_validation_data)
testx_data = pd.read_csv(local_testx_data)
testy_data = pd.read_csv(local_testy_data)

In [10]:
test_data = pd.concat([testy_data, testx_data], axis=1)

In [11]:
test_data

,y,campaign,pdays,previous,no_previous_contact,not_working,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,marital_unknown,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,default_no,default_unknown,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,0,0.018182,1.0,0.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
1,0,0.072727,1.0,0.0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
2,0,0.127273,1.0,0.0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0
3,1,0.000000,1.0,0.0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0
4,0,0.000000,1.0,0.0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6174,0,0.000000,1.0,0.0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0
6175,0,0.000000,1.0,0.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0
6176,1,0.000000,1.0,0.0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0
6177,1,0.018182,1.0,0.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0


# Model training

## Opcao 1 - local:

In [15]:
train_features = train_data.drop(target_col, axis=1)
train_label = pd.DataFrame(train_data[target_col])

In [16]:
dtrain = xgb.DMatrix(train_features, label=train_label)

In [17]:
hyperparams = {
    "max_depth": 5,
    "eta": 0.5,
    "alpha": 2.5,
    "objective": "binary:logistic",
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "min_child_weight": 3
}

num_boost_round = 150
nfold = 3
early_stopping_rounds = 10

In [18]:
# Cross-validate on training data
cv_results = xgb.cv(
    params=hyperparams,
    dtrain=dtrain,
    num_boost_round=num_boost_round,
    nfold=nfold,
    early_stopping_rounds=early_stopping_rounds,
    metrics=["auc"],
    seed=10,
)

In [19]:
# metrics_data = {
#     "binary_classification_metrics": {
#         "validation:auc": {
#             "value": cv_results.iloc[-1]["test-auc-mean"],
#             "standard_deviation": cv_results.iloc[-1]["test-auc-std"]
#         },
#         "train:auc": {
#             "value": cv_results.iloc[-1]["train-auc-mean"],
#             "standard_deviation": cv_results.iloc[-1]["train-auc-std"]
#         },
#     }
# }

In [20]:
print(f"Cross-validated train-auc:{cv_results.iloc[-1]['train-auc-mean']:.2f}")
print(f"Cross-validated validation-auc:{cv_results.iloc[-1]['test-auc-mean']:.2f}")

Cross-validated train-auc:0.81
Cross-validated validation-auc:0.77


In [21]:
cv_results

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.715719,0.002140,0.712067,0.002478
1,0.742467,0.002672,0.736233,0.003434
2,0.753471,0.007740,0.747702,0.004638
3,0.763070,0.004439,0.754373,0.001177
4,0.773070,0.003011,0.764285,0.006123
5,0.778250,0.005665,0.766938,0.003728
6,0.783437,0.002812,0.769662,0.004851
7,0.788049,0.003106,0.771350,0.005044
8,0.790569,0.003532,0.771442,0.006725
9,0.791884,0.003322,0.771694,0.006173


## Opcao 2 - local com experimento:

In [12]:
train_features = train_data.drop(target_col, axis=1)
train_label = pd.DataFrame(train_data[target_col])

In [13]:
dtrain = xgb.DMatrix(train_features, label=train_label)
test_features = test_data.drop(target_col, axis=1)
test_label = pd.DataFrame(test_data[target_col])
dtest = xgb.DMatrix(test_features, label=test_label)

In [18]:
# run_suffix = strftime('%Y-%m-%M-%S', gmtime())
# run_name = f"training-{run_suffix}"

# with Run(experiment_name=experiment_name,
#          run_name=run_name,
#          run_display_name="itau-training",
#          sagemaker_session=sagemaker_session) as run:
#     run.log_parameters(
#         {
#             "train": 0.7,
#             "validate": 0.2,
#             "test": 0.1
#         }
#     )
#     # Log input dataset metadata and output
#     run.log_artifact(name="dataset", value="./data/bank-additional/bank-additional-full.csv", media_type="text/csv", is_output=False)
#     run.log_artifact(name="train-csv", value="./data/train.csv", media_type="text/csv")
#     run.log_artifact(name="validation-csv", value="./data/validation.csv", media_type="text/csv")
#     run.log_artifact(name="test-csv", value="./data/test.csv", media_type="text/csv")

In [18]:
hyperparams = {
    "max_depth": 5,
    "eta": 0.5,
    "alpha": 2.5,
    "objective": "binary:logistic",
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "min_child_weight": 3
}

num_boost_round = 150
early_stopping_rounds = 10

In [20]:
print(f"- EXPERIMENTO: {experiment_name}\n")

# in the production code you need to use the unique ids
run_suffix = strftime('%Y-%m-%d-%H-%M-%S', gmtime())

# Train the model for different max_depth values
for i, d in enumerate([2, 5, 10, 15, 20]):
    hyperparams["max_depth"] = d

    print(f"Fit estimator with max_depth={d}")
    run_name = f"training-{i}-{run_suffix}"
    print(f"run: {run_name}")

    with Run(
        experiment_name=experiment_name,
        run_name=run_name,
        run_display_name=f"max-depth-{d}",
        sagemaker_session=sagemaker_session
    ) as run:
        # Train the model
        model = (
            xgb.train(
                params=hyperparams,
                dtrain=dtrain,
                evals=[(dtrain, 'train'), (dtest, 'eval')],
                num_boost_round=num_boost_round,
                early_stopping_rounds=early_stopping_rounds,
                verbose_eval=0
            )
        )

        # Calculate metrics
        test_auc = roc_auc_score(test_label, model.predict(dtest))
        train_auc = roc_auc_score(train_label, model.predict(dtrain))

        # Log metrics to the run
        run.log_parameters(hyperparams)
        run.log_metric(name="test_auc", value=test_auc, step=d)
        run.log_metric(name="train_auc", value=train_auc, step=d)

        # Log input dataset metadata and output
        # run.log_artifact(name="dataset", value="./data/bank-additional-full.csv", media_type="text/csv", is_output=False)
        run.log_artifact(name="train-csv", value=f"{loca_transformed_path}/train.csv", media_type="text/csv")
        run.log_artifact(name="test-csv", value=f"{loca_transformed_path}/test_x.csv", media_type="text/csv")

        time.sleep(5) # wait until resource tags are propagated to the run

        print(f"Train AUC: {train_auc:.4f} | Test AUC: {test_auc:.4f}")
        print("-------------------------------------")

- EXPERIMENTO: itau-experiment-2024-10-13-18-09-47

Fit estimator with max_depth=2
Train AUC: 0.7859 | Test AUC: 0.7571
------------------------------------
Fit estimator with max_depth=5
Train AUC: 0.8060 | Test AUC: 0.7543
------------------------------------
Fit estimator with max_depth=10
Train AUC: 0.8532 | Test AUC: 0.7502
------------------------------------
Fit estimator with max_depth=15
Train AUC: 0.8703 | Test AUC: 0.7468
------------------------------------
Fit estimator with max_depth=20
Train AUC: 0.8772 | Test AUC: 0.7474
------------------------------------


## Opcao 3 - estimator:

In [23]:
# get training container uri
training_image = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.5-1"
)

print(training_image)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.5-1


In [24]:
s3_data_transformed_prefix = "data/transformed"
train_s3_url = f"s3://{bucket_name}/{s3_prefix}/{s3_data_transformed_prefix}/train"
validation_s3_url = f"s3://{bucket_name}/{s3_prefix}/{s3_data_transformed_prefix}/validation"
test_s3_url = f"s3://{bucket_name}/{s3_prefix}/{s3_data_transformed_prefix}/test"
baseline_s3_url = f"s3://{bucket_name}/{s3_prefix}/{s3_data_transformed_prefix}/baseline"

print(f"{train_s3_url = }")
print(f"{validation_s3_url = }")
print(f"{test_s3_url = }")
print(f"{baseline_s3_url = }")

train_s3_url = 's3://sagemaker-us-east-1-891377318910/workshop_v2/data/transformed/train'
validation_s3_url = 's3://sagemaker-us-east-1-891377318910/workshop_v2/data/transformed/validation'
test_s3_url = 's3://sagemaker-us-east-1-891377318910/workshop_v2/data/transformed/test'
baseline_s3_url = 's3://sagemaker-us-east-1-891377318910/workshop_v2/data/transformed/baseline'


In [25]:
s3_input_train = sagemaker.inputs.TrainingInput(train_s3_url, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(validation_s3_url, content_type='csv')

training_inputs = {'train': s3_input_train, 'validation': s3_input_validation}

In [26]:
train_instance_count = 1
train_instance_type = "ml.m5.xlarge"

# Define where the training job stores the model artifact
output_s3_url = f"s3://{bucket_name}/{s3_prefix}/output"

print(output_s3_url)
%store output_s3_url

s3://sagemaker-us-east-1-891377318910/workshop_v2/output
Stored 'output_s3_url' (str)


In [35]:
job_base_name = "itau-estimator-training"

# Instantiate an XGBoost estimator object
estimator = sagemaker.estimator.Estimator(
    image_uri=training_image,                    # XGBoost algorithm container
    instance_type=train_instance_type,           # type of training instance
    instance_count=train_instance_count,         # number of instances to be used
    role=sagemaker_role,                         # IAM execution role to be used
    max_run=15 * 60,                             # Maximum allowed active runtime
    output_path=output_s3_url,                   # S3 location for saving the training result
    sagemaker_session=sagemaker_session,         # Session object which manages interactions with SageMaker API and AWS services
    base_job_name=job_base_name,                 # Prefix for training job name
    keep_alive_period_in_seconds=60,             # use the warm pool feature
    # use_spot_instances=True,                   # Use spot instances to reduce cost
    # max_wait=30 * 60,                          # Maximum clock time (including spot delays)
)

In [36]:
# define its hyperparameters
estimator.set_hyperparameters(
    num_round=150,             # the number of rounds to run the training
    max_depth=3,               # maximum depth of a tree
    eta=0.5,                   # step size shrinkage used in updates to prevent overfitting
    alpha=2.5,                 # L1 regularization term on weights
    objective="binary:logistic",
    eval_metric="auc",         # evaluation metrics for validation data
    subsample=0.8,             # subsample ratio of the training instance
    colsample_bytree=0.8,      # subsample ratio of columns when constructing each tree
    min_child_weight=3,        # minimum sum of instance weight (hessian) needed in a child
    early_stopping_rounds=10,  # the model trains until the validation score stops improving
    verbosity=1,               # verbosity of printing messages
)

In [38]:
try:
    run_suffix = strftime('%Y-%m-%M-%S', gmtime())
    run_name = f"itau-container-training-{run_suffix}"

    with Run(
         experiment_name=experiment_name,
         run_name=run_name,
         run_display_name="container-training",
         sagemaker_session=sagemaker_session
    ) as run:
        print(f"Estimator: {estimator._current_job_name}")
        estimator.fit(
            training_inputs,
            wait=True,
            logs=True,
        )
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == 'AccessDeniedException':
        print(f"Ignore AccessDeniedException: {e.response['Error']['Message']} because of the slow resource tag auto propagation")
    else:
        raise e

INFO:sagemaker:Creating training-job with name: itau-estimator-training-2024-10-13-22-59-41-453


2024-10-13 22:59:41 Starting - Starting the training job...
2024-10-13 22:59:57 Starting - Preparing the instances for training...
2024-10-13 23:00:22 Downloading - Downloading input data...
2024-10-13 23:00:42 Downloading - Downloading the training image...
2024-10-13 23:01:28 Training - Training image download completed. Training in progress./miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[2024-10-13 23:01:35.069 ip-10-2-106-64.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-10-13 23:01:35.089 ip-10-2-106-64.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-10-13:23:01:35:INFO] Imported framework sagemaker_xgboost_container.training
[2024-10-13:23:01:35:INFO] Failed to parse hyperparameter eval_metric value auc to J

## Option 4 estimator:

In [41]:
train_instance_count = 1
train_instance_type = "ml.m5.xlarge"

s3_data_transformed_prefix = "data/transformed"
train_s3_url = f"s3://{bucket_name}/{s3_prefix}/{s3_data_transformed_prefix}/train"
validation_s3_url = f"s3://{bucket_name}/{s3_prefix}/{s3_data_transformed_prefix}/validation"
test_s3_url = f"s3://{bucket_name}/{s3_prefix}/{s3_data_transformed_prefix}/test"
baseline_s3_url = f"s3://{bucket_name}/{s3_prefix}/{s3_data_transformed_prefix}/baseline"

# Define where the training job stores the model artifact
output_s3_url = f"s3://{bucket_name}/{s3_prefix}"
job_base_name = "itau-estimator-training-best"

print(f"{train_s3_url = }")
print(f"{validation_s3_url = }")
print(f"{test_s3_url = }")
print(f"{baseline_s3_url = }")
print(f"{output_s3_url = }")

train_s3_url = 's3://sagemaker-us-east-1-891377318910/workshop_v2/data/transformed/train'
validation_s3_url = 's3://sagemaker-us-east-1-891377318910/workshop_v2/data/transformed/validation'
test_s3_url = 's3://sagemaker-us-east-1-891377318910/workshop_v2/data/transformed/test'
baseline_s3_url = 's3://sagemaker-us-east-1-891377318910/workshop_v2/data/transformed/baseline'
output_s3_url = 's3://sagemaker-us-east-1-891377318910/workshop_v2'


In [42]:
# get training container uri
training_image = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region_name,
    version="1.7-1"
)

print(training_image)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1


In [43]:
# Instantiate an XGBoost estimator object
estimator = sagemaker.estimator.Estimator(
    image_uri=training_image,             # XGBoost algorithm container
    instance_type=train_instance_type,    # type of training instance
    instance_count=train_instance_count,  # number of instances to be used
    role=sagemaker_role,                  # IAM execution role to be used
    max_run=20 * 60,                      # Maximum allowed active runtime
    output_path=output_s3_url,            # S3 location for saving the training result
    sagemaker_session=sagemaker_session,  # Session object which manages interactions with SageMaker API and AWS services
    base_job_name=job_base_name,          # Prefix for training job name
    keep_alive_period_in_seconds=60,      # use the warm pool feature
    deploy_mode=False,
    # use_spot_instances=True,            # Use spot instances to reduce cost
    # max_wait=30 * 60,                   # Maximum clock time (including spot delays)
)

In [46]:
grid_hiperparametros = {
    'max_depth': IntegerParameter(3, 9),
    'eta': ContinuousParameter(0.1, 0.8),
    'gamma': ContinuousParameter(0.0, 0.8),
    'min_child_weight': IntegerParameter(1, 6),
    'subsample': ContinuousParameter(0.5, 1),
    'alpha': ContinuousParameter(0, 1.0),
    'lambda': ContinuousParameter(0, 0.1),
    'num_round': IntegerParameter(100, 101),
    'colsample_bytree': ContinuousParameter(0, 1),
}

In [47]:
metrics_definitions = [
    {"Name": "train loss", "Regex": "train logloss: ([0-9\\.]+)"},
    {"Name": "train accuracy", "Regex": "train accuracy: ([0-9\\.]+)"},
    {"Name": "validation loss", "Regex": "validation logloss: ([0-9\\.]+)"},
    {"Name": "validation accuracy", "Regex": "validation accuracy: ([0-9\\.]+)"}
]

# Cria o otimizador
estimator_tuner = HyperparameterTuner(
    estimator=estimator,
    objective_metric_name="validation:rmse",
    objective_type="Minimize",
    hyperparameter_ranges=grid_hiperparametros,
    max_jobs=20,
    max_parallel_jobs=8,
    # strategy="Bayesian",
    # early_stopping_type="Auto",
    # base_tuning_job_name="from-idea-to-prod-hpo",
    # metric_definitions=metrics_definitions,
)

In [48]:
s3_input_train = sagemaker.inputs.TrainingInput(train_s3_url, content_type='text/csv')
s3_input_validation = sagemaker.inputs.TrainingInput(validation_s3_url, content_type='text/csv')

In [49]:
%%time
estimator_tuner.fit({'train': s3_input_train, 'validation': s3_input_validation}, wait=True, logs=True)

INFO:sagemaker:Creating hyperparameter tuning job with name: sagemaker-xgboost-241013-2307


......................................................!
CPU times: user 299 ms, sys: 9.24 ms, total: 308 ms
Wall time: 4min 41s


In [51]:
parent_tuning_job_name = estimator_tuner.latest_tuning_job.job_name

parent_results = HyperparameterTuningJobAnalytics(
    sagemaker_session=sagemaker_session,
    hyperparameter_tuning_job_name=parent_tuning_job_name
)

df_parent = parent_results.dataframe()

In [52]:
df_parent = df_parent.sort_values("FinalObjectiveValue", ascending=True)
df_parent.head(5)

,alpha,colsample_bytree,eta,gamma,lambda,max_depth,min_child_weight,num_round,subsample,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
13,0.882522,0.203189,0.220758,0.659159,0.011395,9.0,1.0,101.0,0.687123,sagemaker-xgboost-241013-2307-007-5e9a3315,Completed,0.28325,2024-10-13 23:08:41+00:00,2024-10-13 23:10:21+00:00,100.0
0,0.986819,0.218964,0.396115,0.772141,0.016141,3.0,6.0,100.0,0.565662,sagemaker-xgboost-241013-2307-020-dda6d29c,Completed,0.28417,2024-10-13 23:11:41+00:00,2024-10-13 23:12:11+00:00,30.0
16,0.598965,0.296738,0.339805,0.270576,0.039365,4.0,4.0,100.0,0.878519,sagemaker-xgboost-241013-2307-004-7f639bc9,Completed,0.28417,2024-10-13 23:08:42+00:00,2024-10-13 23:10:17+00:00,95.0
3,0.148648,0.865727,0.190530,0.445350,0.085885,4.0,5.0,101.0,0.553581,sagemaker-xgboost-241013-2307-017-9e64f72d,Completed,0.28499,2024-10-13 23:11:25+00:00,2024-10-13 23:11:59+00:00,34.0
7,0.356018,0.107690,0.414163,0.129009,0.056781,8.0,6.0,100.0,0.526816,sagemaker-xgboost-241013-2307-013-8329dfb1,Completed,0.28547,2024-10-13 23:10:48+00:00,2024-10-13 23:11:17+00:00,29.0


In [53]:
estimator_tuner.stop_tuning_job()

INFO:sagemaker:Stopping tuning job: sagemaker-xgboost-241013-2307
INFO:sagemaker:Tuning job: sagemaker-xgboost-241013-2307 is already stopped or not running.


In [57]:
best_training_job_name = estimator_tuner.best_training_job()
print(f"Melhor Job: {best_training_job_name}")

Melhor Job: sagemaker-xgboost-241013-2307-007-5e9a3315


In [58]:
# Descrevendo o melhor job de treinamento para obter os hiperparâmetros
best_training_job_info = sagemaker_session.describe_training_job(job_name=best_training_job_name)

In [59]:
best_hyperparameters = best_training_job_info['HyperParameters']

In [66]:
print("Melhores Hiperparâmetros:\n")
pprint(best_hyperparameters, indent=4)

Melhores Hiperparâmetros:

{   '_tuning_objective_metric': 'validation:rmse',
    'alpha': '0.8825218857263171',
    'colsample_bytree': '0.20318899339072938',
    'eta': '0.2207575226741727',
    'gamma': '0.6591589238640063',
    'lambda': '0.011394837123698676',
    'max_depth': '9',
    'min_child_weight': '1',
    'num_round': '101',
    'subsample': '0.6871233173328715'}


In [67]:
# Set dos melhores hiperparâmetros
estimator.set_hyperparameters(**best_hyperparameters)

In [68]:
# Treinando a versão final do modelo com os melhores hiperparâmetros
train_inputs = {"train": s3_input_train, "validation": s3_input_validation}
estimator.fit(inputs=train_inputs, job_name=job_base_name, wait=True, logs=True)

INFO:sagemaker:Creating training-job with name: itau-estimator-training-best


2024-10-13 23:21:05 Starting - Starting the training job...
2024-10-13 23:21:20 Starting - Preparing the instances for training...
2024-10-13 23:22:05 Downloading - Downloading the training image......
2024-10-13 23:22:56 Training - Training image download completed. Training in progress..[2024-10-13 23:23:06.975 ip-10-0-80-40.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-10-13 23:23:06.999 ip-10-0-80-40.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-10-13:23:23:07:INFO] Imported framework sagemaker_xgboost_container.training
[2024-10-13:23:23:07:INFO] Failed to parse hyperparameter _tuning_objective_metric value validation:rmse to Json.
Returning the value itself
[2024-10-13:23:23:07:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-13:23:23:07:INFO] Running XGBoost Sagemaker in algorithm mode
[2024-10-13:23:23:07:INFO] Determined 0 GPU(s) available on the instance.
[2024-10-13:23:23:07:INFO] Determined del

# Evaluate the Model

In [69]:
if "estimator" in globals():
    if estimator._current_job_name:
        training_job_name = estimator._current_job_name
        print("Job name from estimator:", training_job_name)
else:
    training_job_name = job_base_name
    print("Job name from training_job_name:", training_job_name)

Job name from estimator: itau-estimator-training-best


In [70]:
metrics = None

while not metrics:
    metrics = client_sagemaker.describe_training_job(TrainingJobName=training_job_name).get("FinalMetricDataList")
    if not metrics:
        print(f"Training job {training_job_name} hasn't finished yet!")
        time.sleep(5)

# train_auc = [m['Value'] for m in metrics if m['MetricName'] == 'train:auc']
# validate_auc = float([m['Value'] for m in metrics if m['MetricName'] == 'validation:auc'][0])
# print(f"Train-auc:{train_auc:.2f}, Validate-auc:{validate_auc:.2f}")

In [71]:
for item in metrics:
    print(item)

{'MetricName': 'train:rmse', 'Value': 0.2779900133609772, 'Timestamp': datetime.datetime(2024, 10, 13, 23, 23, 11, tzinfo=tzlocal())}
{'MetricName': 'validation:rmse', 'Value': 0.28325000405311584, 'Timestamp': datetime.datetime(2024, 10, 13, 23, 23, 11, tzinfo=tzlocal())}


In [72]:
print(estimator.model_data)

s3://sagemaker-us-east-1-891377318910/workshop_v2/itau-estimator-training-best/output/model.tar.gz


In [87]:
bucket = estimator.model_data.replace("s3://", "").split("/")[0]
key = estimator.model_data.replace(f"s3://{bucket}/", "")

print(f"Bucket - modelo: {bucket}")
print(f"Key - modelo:    {key}")

client_s3.download_file(Bucket=bucket, Key=key, Filename="./model/model.tar.gz")

Bucket - modelo: sagemaker-us-east-1-891377318910
Key - modelo:    workshop_v2/itau-estimator-training-best/output/model.tar.gz


## Modelo

In [70]:
# !tar -xvzf ./model.tar.gz ./model

In [84]:
# # Carrega o modelo
# with open('model.tar.gz', 'rb') as file:
#     xgboost_model = pickle.load(file)

In [89]:
with tarfile.open('./model/model.tar.gz', 'r:gz') as file:
    file.extractall(path="./model")

### Registrar modelo - deployable models

In [21]:
# model_name = f"{training_job_name}-model"
# print("Nome Modelo:", model_name)

# info = client_sagemaker.describe_training_job(TrainingJobName=training_job_name)
# model_data = info["ModelArtifacts"]["S3ModelArtifacts"]

# primary_container = {"Image": training_image, "ModelDataUrl": model_data}

# create_model_response = client_sagemaker.create_model(
#     ModelName=model_name,
#     ExecutionRoleArn=sagemaker_role,
#     PrimaryContainer=primary_container
# )

# print(create_model_response["ModelArn"])

### Registrar modelo - registered models

In [137]:
model_package_group_name = "itau-model-xgboost" + strftime('%Y-%m-%d-%H-%M-%S', gmtime())

model_package_group_input_dict = {
     "ModelPackageGroupName": model_package_group_name,
     "ModelPackageGroupDescription": "Sample model package group"
}

create_model_package_group_response = client_sagemaker.create_model_package_group(**model_package_group_input_dict)
print('ModelPackageGroup Arn : {}'.format(create_model_package_group_response['ModelPackageGroupArn']))

ModelPackageGroup Arn : arn:aws:sagemaker:us-east-1:891377318910:model-package-group/testando-model-1728867288


In [138]:
model_url = estimator.model_data

modelpackage_inference_specification = {
    "InferenceSpecification": {
      "Containers": [
         {
            "Image": training_image,
            "ModelDataUrl": model_url
         }
      ],
      "SupportedContentTypes": ["text/csv"],
      "SupportedResponseMIMETypes": ["text/csv"]
    }
}

# Alternatively, you can specify the model source like this:
# modelpackage_inference_specification["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]=model_url

create_model_package_input_dict = {
    "ModelPackageGroupName": model_package_group_name,
    "ModelPackageDescription": "Model to detect 3 different types of irises (Setosa, Versicolour, and Virginica)",
    "ModelApprovalStatus": "PendingManualApproval"
}

create_model_package_input_dict.update(modelpackage_inference_specification)

In [139]:
create_model_package_response = client_sagemaker.create_model_package(**create_model_package_input_dict)
model_package_arn = create_model_package_response["ModelPackageArn"]

print('ModelPackage Version ARN : {}'.format(model_package_arn))

ModelPackage Version ARN : arn:aws:sagemaker:us-east-1:891377318910:model-package/testando-model-1728867288/1


## Modelo local

In [92]:
model = xgb.Booster()
model.load_model("./model/xgboost-model")

In [93]:
# Extrai os atrobutos e labels
X_test_xgb = test_data.drop(columns=['y'])
y_test_xgb = test_data['y']

In [94]:
# Converte o dataframe para objeto DMatrix
dtest_xgb = xgb.DMatrix(X_test_xgb)

In [95]:
# Faz as previsões com o modelo XGBoost
predictions_xgboost = model.predict(dtest_xgb)
predictions_xgboost_rounded = np.round(predictions_xgboost)

In [96]:
test_data['predictions'] = predictions_xgboost_rounded

In [97]:
# Calcula as métricas de avaliação do XGBoost com dados de teste
xgboost_metrics = {
    'MSE (XGBoost)': mean_squared_error(y_test_xgb, predictions_xgboost_rounded),
    'MAE (XGBoost)': mean_absolute_error(y_test_xgb, predictions_xgboost_rounded),
    'RMSE (XGBoost)': np.sqrt(mean_squared_error(y_test_xgb, predictions_xgboost_rounded))
}

In [98]:
print(xgboost_metrics)

{'MSE (XGBoost)': 0.10163456870043697, 'MAE (XGBoost)': 0.10163456870043697, 'RMSE (XGBoost)': 0.3188017702278909}


In [100]:
# Cria o dataframe com dados reais e previstos
xgboost_results_df = pd.DataFrame({'Real': y_test_xgb, 'Previsto': predictions_xgboost_rounded})
xgboost_results_df.tail(10)

,Real,Previsto
6169,0,0.0
6170,0,0.0
6171,1,1.0
6172,0,0.0
6173,0,0.0
6174,0,0.0
6175,0,0.0
6176,1,0.0
6177,1,0.0
6178,0,0.0


## Validate model

In [49]:
dtime = strftime('%Y-%m-%d-%H-%M-%S', gmtime())

train_instance_type = "ml.m5.xlarge"
job_name = "itau-best-xgboost-1"
transform_s3_url = f"s3://{bucket_name}/{s3_prefix}/data/output"
model_name = "itau-model-transform-2024-10-14-00-03-24" # f"itau-model-xgboost-{dtime}"
model = f"itau-model-transform-{dtime}"
base_transform_job_name = "itau-demo-trasform"
training_job_name = "itau-estimator-training-best"

transform_job_name = f"{base_transform_job_name}-{dtime}"

In [50]:
print(f"training_job_name:       {training_job_name}")
print(f"transform_s3_url:        {transform_s3_url}")
print(f"model_name:              {model_name}")
print(f"base_transform_job_name: {base_transform_job_name}")
# print(f"{test_s3_url = }")

print(f"{transform_job_name = }")

training_job_name:       itau-estimator-training-best
transform_s3_url:        s3://sagemaker-us-east-1-891377318910/workshop_v2/data/transformed
model_name:              itau-model-transform-2024-10-14-00-03-24
base_transform_job_name: itau-demo-trasform
transform_job_name = 'itau-demo-trasform-2024-10-15-00-29-12'


### Option 1: create a batch transformer from the trained estimator
You can use [`EstimatorBase.transformer()`](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.transformer) to create a transformer for an estimator:

In [41]:
# transformer = estimator.transformer(
#     instance_count=1,
#     instance_type=train_instance_type,
#     accept="text/csv",
#     assemble_with="Line",
#     max_concurrent_transforms=2,
#     role=sagemaker_role,
#     output_path=transform_s3_url,
#     model_name=model_name,
# )

### Option 2: load a model from a training job

In [113]:
# model = sagemaker_session.create_model_from_job(
#     training_job_name=training_job_name,
#     name=model_name,
# )

In [51]:
transformer = sagemaker.transformer.Transformer(
    model_name=model_name,
    instance_count=1,
    instance_type=train_instance_type,
    accept="text/csv",
    assemble_with="Line",
    output_path=transform_s3_url,
    base_transform_job_name=base_transform_job_name,
    sagemaker_session=sagemaker_session,
)

### Run transform job

In [52]:
# transformer.transform(
#     data=f"{test_s3_url}/test_x.csv",
#     content_type="text/csv",
#     split_type="Line",
#     job_name=transform_job_name,
#     wait=True,
# )

In [53]:
# s3_data_transformed_prefix = "data/transformed"
# test_s3_url = f"s3://{bucket_name}/{s3_prefix}/{s3_data_transformed_prefix}/test"
test_s3_url

's3://sagemaker-us-east-1-891377318910/workshop_v2/data/transformed/test'

In [54]:
run_suffix = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
run_name = f"itau-batch-transform-{run_suffix}"
print(f"Run name: {run_name}")

try:
    with Run(
        experiment_name=experiment_name,
        run_name=run_name,
        run_display_name="itau-batch-transform",
        sagemaker_session=sagemaker_session
    ) as run:
        transformer.transform(
            data=f"{test_s3_url}/test_x.csv",
            content_type="text/csv",
            split_type="Line",
            job_name=transform_job_name,
            wait=True,
            # experiment_config=experiment_config,
        )
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == 'AccessDeniedException':
        print(f"Ignore AccessDeniedException: {e.response['Error']['Message']} because of the slow resource tag auto propagation")
    else:
        raise e

Run name: itau-batch-transform-2024-10-15-00-29-17


INFO:sagemaker:Creating transform job with name: itau-demo-trasform-2024-10-15-00-29-12


......................................[2024-10-15:00:35:34:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-15:00:35:34:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-15:00:35:34:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
[2024-10-15:00:35:34:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-15:00:35:34:INFO] No GPUs detected (normal if no gpus installed)
[2024-10-15:00:35:34:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
     

In [ ]:
while client_sagemaker.describe_transform_job(
    TransformJobName=transformer._current_job_name
)["TransformJobStatus"] != "Completed":
    time.sleep(15)
    print(f"Wait until {transformer._current_job_name} completed")

print("Finished!")

In [57]:
print(transformer.output_path)
print(f"Job name: {transformer._current_job_name}")

s3://sagemaker-us-east-1-891377318910/workshop_v2/data/transformed
Job name: itau-demo-trasform-2024-10-15-00-29-12
